In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/human-stress-prediction/Stress.csv


In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split




In [3]:
stress_data_path = '../input/human-stress-prediction/Stress.csv'
stress_data = pd.read_csv(stress_data_path)
stress_data.head()

,subreddit,post_id,sentence_range,text,label,confidence,social_timestamp
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",1,0.8,1521614353
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",0,1.0,1527009817
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,1,0.8,1535935605
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",1,0.6,1516429555
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1,0.8,1539809005


In [4]:
longest_post = max(stress_data.text, key=len)
len(longest_post.split()) #nr of words in the longest post

292

In [5]:
post_lengths = stress_data.text.apply(len) #lengths of all posts

post_lengths.mean() #avg length
stress_data.text

0       He said he had not felt that way before, sugge...
1       Hey there r/assistance, Not sure if this is th...
2       My mom then hit me with the newspaper and it s...
3       until i met my new boyfriend, he is amazing, h...
4       October is Domestic Violence Awareness Month a...
                              ...                        
2833    * Her, a week ago: Precious, how are you? (I i...
2834    I don't have the ability to cope with it anymo...
2835    In case this is the first time you're reading ...
2836    Do you find this normal? They have a good rela...
2837    I was talking to my mom this morning and she s...
Name: text, Length: 2838, dtype: object

# **Text Preprocessing**

In [6]:

#removing special characters
def remove_spec(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

clean_posts = list(map(remove_spec, stress_data.text))


In [7]:
#spellchecking???

In [8]:
#stopwords are words that mostly have grammatical meaning, which we can exclude

# List of sentences to be vectorized
lines = clean_posts

# Removing stop words
stop_words = set(stopwords.words('english'))
lines_without_stopwords = []
for line in lines:
    words = line.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    lines_without_stopwords.append(' '.join(filtered_words))

lines = lines_without_stopwords

# Lemmatization
#wordnet_lemmatizer = WordNetLemmatizer()
#lines_with_lemmas = []
#for line in lines:
 #   words = line.split()
  #  lemmatized_words = [wordnet_lemmatizer.lemmatize(word) for word in words]
   # lines_with_lemmas.append(' '.join(lemmatized_words))

#lines = lines_with_lemmas
clean_posts = lines

# **Splitting Data**

In [48]:
X = clean_posts #attributes according to which we classify
y = stress_data.label #class

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=4)



In [54]:
X = clean_posts #attributes according to which we classify
y = stress_data.label #class
# Convert X and y to NumPy arrays

# Initialize the stratified k-fold cross-validation object
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
split_data = skf.split(X, y)

# **Bag of Words**

In [34]:
# create the transform
tf_vectorizer=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,1))

# tokenize and build vocab
bow_train = tf_vectorizer.fit_transform(X_train)
# encode document
bow_test = tf_vectorizer.transform(X_test)
# summarize
#print(sorted(vectorizer.vocabulary_))
print('BOW train: ',bow_train.shape)
print('BOW test: ',bow_test.shape)

BOW train:  (2128, 5360)
BOW test:  (710, 5360)


In [12]:
print(len(vectorizer.idf_))

# summarize encoded vector
print(vectorized_posts[0].shape)
print(vectorized_posts[0].toarray())


NameError: name 'vectorizer' is not defined

In [ ]:
#print(len(vectorized_posts))
#print(len(stress_data))


# **Bag of N-Grams**

In [35]:
# create the transform
tf_vectorizer2 = TfidfVectorizer(analyzer='word', ngram_range=(7, 7)) #instead of separate words treat them as 7-grams, groups of len 7
# tokenize and build vocab
bon_train = tf_vectorizer2.fit_transform(X_train)

# encode document
bon_test = tf_vectorizer2.transform(X_test)
# summarize
#print(sorted(vectorizer.vocabulary_))
print('BON train: ',bow_train.shape)
print('BON test: ',bow_test.shape)



BON train:  (2128, 5360)
BON test:  (710, 5360)


# **Word2Vec**

In [14]:
vectorized_w2v = [gensim.utils.simple_preprocess(text) for text in clean_posts]
len(vectorized_w2v)



2838

In [15]:
# initialize the model
#w2v = Word2Vec(clean_posts, min_count=1, vector_size=3)

# Model parameters
w2v=gensim.models.Word2Vec(window=5, min_count=1,vector_size=200, workers=4, sg=0)

# train the model
w2v.build_vocab(vectorized_w2v)
w2v.train(vectorized_w2v, total_examples=w2v.corpus_count, epochs=w2v.epochs)
1
# save the trained model
#w2v.save("./responses.model")
#print(w2v.wv['plastic']) 

#print((w2v.wv.key_to_index)) 


1

In [16]:
# Initialize an empty list to store post vectors
post_vectors = []

# Iterate through each tokenized list of words (each post)
for post in vectorized_w2v:
    # Initialize an empty list to store word vectors for this post
    word_vectors_post = []
    
    # Iterate through the words in the post
    for word in post:
        # Check if the word exists in the vocabulary
            # Retrieve the word vector from the model
            word_vector = w2v.wv[word]
            # Append the word vector to the list of word vectors for this post
            word_vectors_post.append(word_vector)
    
    
        # Calculate the average of word vectors for this post
    post_vector = np.sum(word_vectors_post, axis=0)  # You can also use np.sum() instead of np.mean() to get the sum
    # Append the post vector to the list of post vectors
    post_vectors.append(post_vector)
    

# Convert the list of post vectors to a numpy array
post_vectors_array = np.array(post_vectors)

vectorized_posts = post_vectors_array
# Now, post_vectors_array contains the vector representations for each post,
# where each row corresponds to a post and each column corresponds to a dimension of the post vector






# **Naive Bayes**

* 1. naive bayes with stratified kfold splitting and bag of words 59% 
* nb with train test split and bag of ngrams 51%

In [56]:
# Initialize the Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Initialize variables to store the total accuracy across all folds
total_accuracy = 0.0

# Iterate over the folds
for train_index, test_index in split_data:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Fit the model on the training data
    gnb.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = gnb.predict(X_test)

    # Calculate accuracy for this fold
    fold_accuracy = accuracy_score(y_test, y_pred)

    # Accumulate accuracy for all folds
    total_accuracy += fold_accuracy

    # Print accuracy for this fold
    print("Accuracy for this fold:", fold_accuracy)

# Calculate average accuracy across all folds
average_accuracy = total_accuracy / skf.n_splits

print("Average Accuracy:", average_accuracy)



AttributeError: 'list' object has no attribute 'shape'

In [38]:
# initialize the naive bayes classifier
gnb1 = GaussianNB()
gnb2 = GaussianNB()

#fitting the nb for bag of words
gnb_bow=gnb1.fit(bow_train.toarray(), y_train)
print(gnb_bow)
#fitting the nb for tfidf features
gnb_bon=gnb2.fit(bon_train.toarray(), y_train)
print(gnb_bon)


GaussianNB()
GaussianNB()


In [39]:
#Predicting the model for bag of words
gnb_bow_predict = gnb_bow.predict(bow_test.toarray())

#Predicting the model for bag of ngrams
gnb_bon_predict = gnb_bon.predict(bon_test.toarray())


In [40]:
#Accuracy score for bag of words
gnb_bow_score=accuracy_score(y_test,gnb_bow_predict)
print("gnb_bow_score :",gnb_bow_score)
#Accuracy score for tfidf features
gnb_bon_score=accuracy_score(y_test,gnb_bon_predict)
print("gnb_bon_score :",gnb_bon_score)

gnb_bow_score : 0.5309859154929577
gnb_bon_score : 0.48169014084507045


In [41]:
# calculate the number of mislabeled points
num_mislabeled = (y_test != y_pred).sum()
accuracy = num_mislabeled/len(X_test)*100.0
 

print("Number of mislabeled points out of a total %d points: %d" % (len(X_test), num_mislabeled))
print("Accuracy: %d " % (accuracy))


NameError: name 'y_pred' is not defined

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB

X = vectorized_posts #attributes according to which we classify
y = stress_data.label #class

# split data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0)

n_splits = 5

# Initialize the StratifiedKFold object
stratified_kfold = StratifiedKFold(n_splits=n_splits,shuffle=True, random_state=0)
# initialize the naive bayes classifier
gnb = GaussianNB()
acc = 0
x = 0
n =0 
# Iterate over the splits
for train_index, test_index in stratified_kfold.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # fit the model on the training data
    gnb.fit(X_train.toarray(), y_train)
    #make a prediction on the test data
    y_pred = gnb.predict(X_test.toarray())
    # calculate the number of mislabeled points
    num_mislabeled = (y_test != y_pred).sum()
    n += num_mislabeled
    x+=X_test.shape[0]
    accuracy = num_mislabeled/X_test.shape[0]*100.0
    acc+=accuracy
    

print("Number of mislabeled points out of a total %d points: %d" % (x, n))
print("Accuracy: %d " % (acc/5))